In [1]:
!pip install -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 35.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [4]:
dataset = datasets.load_dataset('bentrevett/multi30k')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
train_data , valid_data , test_data = (
    dataset['train'] ,
    dataset['validation'] ,
    dataset['test']
)

In [6]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [7]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

In [8]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example , fn_kwargs= fn_kwargs)
valid_data = valid_data.map(tokenize_example , fn_kwargs= fn_kwargs)
test_data = test_data.map(tokenize_example , fn_kwargs= fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [10]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [11]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

In [12]:
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [13]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [15]:
def collate_fn(batch):
    batch_en_ids = [example["en_ids"] for example in batch]
    batch_de_ids = [example["de_ids"] for example in batch]
    batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
    batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
    batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
    return batch

In [16]:
def get_data_loader(dataset , batch_size , shuffle = False):
  data_loader = torch.utils.data.DataLoader(
      dataset = dataset ,
      batch_size = batch_size ,
      collate_fn = collate_fn ,
      shuffle = shuffle
  )
  return data_loader

In [17]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size)
test_data_loader = get_data_loader(test_data, batch_size)

In [18]:
class Encoder(nn.Module):
  def __init__(
        self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout
    ):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim,)
    self.fc = nn.Linear(encoder_hidden_dim , decoder_hidden_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self , src):
    embedded = self.dropout(self.embedding(src))
    outputs , hidden = self.rnn(embedded)
    # hidden = [n layers , batch size, hidden dim]
    # outputs = [num_steps , batch size, hidden dim]
    hidden = torch.tanh(
        self.fc(hidden[-1,:,:])
    )
    # outputs = [src length , batch size , encoder hidden dim ]
    # hidden = [batch size , decoder hidden dim]
    return outputs , hidden

In [19]:
class Attention(nn.Module):
  def __init__(self , encoder_hidden_dim , decoder_hidden_dim):
    super().__init__()
    self.attn_fc = nn.Linear(
        encoder_hidden_dim + decoder_hidden_dim , decoder_hidden_dim
    )
    self.v_fc = nn.Linear (decoder_hidden_dim , 1 , bias = False)

  def forward(self, hidden , encoder_outputs):
    batch_size = encoder_outputs.shape[1]
    src_length = encoder_outputs.shape[0]
    hidden = hidden.unsqueeze(1).repeat(1, src_length, 1)
    # hidden = [batch size,src_length ,decoder hidden dim]
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    # encoder_outputs = [batch size , src length,  encoder hidden dim]

    energy = torch.tanh(self.attn_fc(torch.cat((hidden , encoder_outputs) , dim = 2)))
    # energy = [batch size , src length , decoder hidden dim]
    attention = self.v_fc(energy).squeeze(2)
    # attention = [batch size , src length]
    return torch.softmax(attention , dim =1)

In [20]:
class Decoder(nn.Module):
  def __init__(self , output_dim , embedding_dim , encoder_hidden_dim , decoder_hidden_dim , dropout ,
               attention ):
    super().__init__()
    self.output_dim = output_dim
    self.attention = attention
    self.embedding = nn.Embedding(output_dim, embedding_dim)
    self.rnn = nn.GRU(encoder_hidden_dim  + embedding_dim, decoder_hidden_dim)
    self.fc_out = nn.Linear(
            encoder_hidden_dim  + decoder_hidden_dim + embedding_dim, output_dim
        )
    self.dropout = nn.Dropout(dropout)

  def forward(self , input , hidden , encoder_outputs):
    # input = [batch size]
    # hidden = [batch size, decoder hidden dim]
    # encoder_outputs = [src length, batch size, encoder hidden dim]
    input = input.unsqueeze(0)
    # input = [ 1 , batchsize]
    embedded = self.dropout(self.embedding(input))

    a = self.attention(hidden ,encoder_outputs)
    # a = [batch size , src length]
    a  = a.unsqueeze(1)
    # a = [batch size , 1 , src length]
    encoder_outputs = encoder_outputs.permute(1,0,2)
    # encoder_outputs = [batch size, src length, encoder hidden dim ]



    weighted = torch.bmm(a, encoder_outputs)
    # weighted = [batch size, 1, encoder hidden dim ]
    weighted = weighted.permute(1,0,2)
    # weighted = [1,batch size,  encoder hidden dim ]
    rnn_input = torch.cat((embedded, weighted), dim=2)
    # rnn_input = [1, batch size, (encoder hidden dim * 2) + embedding dim]
    output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
    # output = [seq length, batch size, decoder hid dim * n directions]
    # hidden = [n layers * n directions, batch size, decoder hid dim]
    # seq len, n layers and n directions will always be 1 in this decoder, therefore:
    # output = [1, batch size, decoder hidden dim]
    # hidden = [1, batch size, decoder hidden dim]
    # this also means that output == hidden
    assert ( output == hidden).all()
    embedded = embedded.squeeze(0)
    output = output.squeeze(0)
    weighted = weighted.squeeze(0)
    prediction = self.fc_out(torch.cat((output , weighted , embedded) , dim = 1))
    return prediction , hidden.squeeze(0) , a.squeeze(1)

In [21]:
class Seq2Seq(nn.Module):
  def __init__(self , encoder , decoder , device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self , src , trg , teacher_forcing_ratio):
    batch_size = trg.shape[1]
    trg_length = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim

    outputs = torch.zeros(trg_length , batch_size , trg_vocab_size).to(self.device)
    # output = [trg_length , batch size, output dim]
    encoder_outputs, hidden = self.encoder(src)
    input = trg[0]
    for t in range(1 , trg_length):
      output , hidden , _  = self.decoder(input , hidden , encoder_outputs)
      # output = [batch size, output dim]
      # print(f'output : {output.shape}')
      outputs[t] = output
      teacher_force = random.random() < teacher_forcing_ratio
      # get the highest predicted token from our predictions
      top1 = output.argmax(1)
      # print(f'top1 : {top1}')
      # print(top1.shape)
      # print(f'trg : {trg[t]}')
      input = trg[t] if teacher_force else top1
      # input = [batch size]
    return outputs

In [22]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attention = Attention(encoder_hidden_dim, decoder_hidden_dim)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    decoder_dropout,
    attention,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [23]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512)
    (fc): Linear(in_features=512, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1024, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [24]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [25]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["de_ids"].to(device)
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        output_dim = output.shape[-1]
        # output = [trg length, batch size, trg vocab size]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [26]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [27]:
n_epochs = 20
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut2-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  5%|▌         | 1/20 [01:10<22:12, 70.13s/it]

	Train Loss:   5.050 | Train PPL: 156.007
	Valid Loss:   4.894 | Valid PPL: 133.537


 10%|█         | 2/20 [02:22<21:27, 71.55s/it]

	Train Loss:   4.372 | Train PPL:  79.215
	Valid Loss:   4.722 | Valid PPL: 112.439


 15%|█▌        | 3/20 [03:34<20:22, 71.89s/it]

...

	Train Loss:   1.075 | Train PPL:   2.929
	Valid Loss:   3.712 | Valid PPL:  40.918


 95%|█████████▌| 19/20 [23:01<01:12, 72.92s/it]

	Train Loss:   1.031 | Train PPL:   2.804
	Valid Loss:   3.757 | Valid PPL:  42.839


100%|██████████| 20/20 [24:13<00:00, 72.67s/it]

	Train Loss:   0.967 | Train PPL:   2.631
	Valid Loss:   3.793 | Valid PPL:  44.398


In [28]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            de_tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            de_tokens = [token for token in sentence]
        if lower:
            de_tokens = [token.lower() for token in de_tokens]
        de_tokens = [sos_token] + de_tokens + [eos_token]
        ids = de_vocab.lookup_indices(de_tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        encoder_outputs, hidden = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        attentions = torch.zeros(max_output_length, 1, len(ids))
        for i in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, attention = model.decoder(
                inputs_tensor, hidden, encoder_outputs
            )
            attentions[i] = attention
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        en_tokens = en_vocab.lookup_tokens(inputs)
    return en_tokens, de_tokens, attentions[: len(en_tokens) - 1]

In [29]:
sentence = test_data[20]["de"]
expected_translation = test_data[20]["en"]

sentence, expected_translation

('Leute, die vor einem Gebäude stehen.',
 'People standing outside of a building.')

In [30]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [31]:
translation

(['<sos>',
  'people',
  'gathered',
  'in',
  'front',
  'of',
  'a',
  'building',
  '.',
  '<eos>'],
 ['<sos>',
  'leute',
  ',',
  'die',
  'vor',
  'einem',
  'gebäude',
  'stehen',
  '.',
  '<eos>'],
 tensor([[[1.7553e-03, 9.4707e-01, 4.4878e-02, 7.6550e-04, 3.0385e-04,
           1.1740e-04, 2.3207e-03, 1.4020e-03, 7.4058e-04, 6.4450e-04]],
 
         [[3.9945e-03, 6.1081e-02, 3.3989e-01, 1.1183e-01, 8.1709e-02,
           3.9375e-03, 4.1747e-02, 1.6723e-01, 1.3462e-01, 5.3963e-02]],
 
         [[9.5201e-03, 3.3043e-02, 2.2239e-02, 6.3427e-02, 3.5441e-01,
           2.1511e-02, 2.2347e-02, 1.8637e-01, 1.8760e-01, 9.9534e-02]],
 
         [[5.1752e-03, 4.5445e-03, 9.3576e-04, 1.0064e-03, 1.0633e-01,
           6.9773e-02, 1.2944e-01, 3.3930e-01, 2.5656e-01, 8.6934e-02]],
 
         [[8.9238e-04, 1.7849e-03, 1.5288e-04, 3.6522e-05, 1.3147e-02,
           3.4390e-02, 1.8686e-01, 5.7288e-01, 1.5134e-01, 3.8504e-02]],
 
         [[1.9007e-03, 4.4386e-03, 2.8514e-04, 2.8340e-05, 5.757